In [1]:
from pathlib import Path

import altair as alt
import pandas as pd
import xarray as xr
from ruamel.yaml import YAML

yaml = YAML()

# Read the data for all regions and projects

In [2]:
def load_single_file(folder, metadata, custom_project_name):
    filename = folder / Path(metadata["filename"]).name
    ds = xr.open_dataset(filename).drop(["lat", "lon"])

    da = ds[metadata["short_name"]]
    da["project"] = custom_project_name
    da["dataset"] = metadata["dataset"]
    da["ensemble"] = metadata["ensemble"]

    da = da.drop(["ensemble_member", "ensemble_member_id"], errors="ignore")  # UKCP
    if custom_project_name == "cordex_cpm3":
        da["ensemble"] = "inconsistent"

    df = da.to_dataframe().set_index(["project", "dataset", "ensemble"], append=True)
    return df


def load_data(region, exp, project):
    folder = Path(
        f"/home/peter/eucp-project/tom_data/preproc/{region}/boxplots/box_{exp}_{project}"
    )

    with open(folder / "metadata.yml", "r") as f:
        metadata = yaml.load(f)

    return pd.concat(
        [load_single_file(folder, meta, project) for meta in metadata.values()]
    )

In [3]:
regions = ["Alpine", "Alps_NE", "Alps_NW", "Alps_SE", "Alps_SW", "DE_S", "FRA_S"]
projects = [
    "CMIP5",
    "CMIP6",
    "cordex_cpm",
    "cordex_cpm2",
    "cordex_cpm3",
    "CORDEX-EUR11",
    "UKCP-GCM",
    "UKCP-RCM",
]

all_difs = []
for region in regions:
    for project in projects:
        print(f"Reading data for {region}, {project}")
        exp = "SSP585" if project == "CMIP6" else "rcp85"

        hist = load_data(region, "hist", project)
        future = load_data(region, exp, project)

        difference = (future - hist) / hist * 100
        difference["region"] = region
        all_difs.append(difference)

data = pd.concat(all_difs).reset_index()
data

Reading data for Alpine, CMIP5
Reading data for Alpine, CMIP6
Reading data for Alpine, cordex_cpm
Reading data for Alpine, cordex_cpm2
Reading data for Alpine, cordex_cpm3
Reading data for Alpine, CORDEX-EUR11
Reading data for Alpine, UKCP-GCM
Reading data for Alpine, UKCP-RCM
Reading data for Alps_NE, CMIP5
Reading data for Alps_NE, CMIP6
Reading data for Alps_NE, cordex_cpm
Reading data for Alps_NE, cordex_cpm2
Reading data for Alps_NE, cordex_cpm3
Reading data for Alps_NE, CORDEX-EUR11
Reading data for Alps_NE, UKCP-GCM
Reading data for Alps_NE, UKCP-RCM
Reading data for Alps_NW, CMIP5
Reading data for Alps_NW, CMIP6
Reading data for Alps_NW, cordex_cpm
Reading data for Alps_NW, cordex_cpm2
Reading data for Alps_NW, cordex_cpm3
Reading data for Alps_NW, CORDEX-EUR11
Reading data for Alps_NW, UKCP-GCM
Reading data for Alps_NW, UKCP-RCM
Reading data for Alps_SE, CMIP5
Reading data for Alps_SE, CMIP6
Reading data for Alps_SE, cordex_cpm
Reading data for Alps_SE, cordex_cpm2
Reading dat

,season_number,project,dataset,ensemble,pr,region
0,0,CMIP5,ACCESS1-0,r1i1p1,1.584215,Alpine
1,1,CMIP5,ACCESS1-0,r1i1p1,12.019885,Alpine
2,2,CMIP5,ACCESS1-0,r1i1p1,-9.247226,Alpine
3,3,CMIP5,ACCESS1-0,r1i1p1,-0.685971,Alpine
4,0,CMIP5,ACCESS1-3,r1i1p1,3.111616,Alpine
...,...,...,...,...,...,...
3803,3,UKCP-RCM,land-rcm,13,-11.715302,FRA_S
3804,0,UKCP-RCM,land-rcm,15,8.348953,FRA_S
3805,1,UKCP-RCM,land-rcm,15,-7.254844,FRA_S
3806,2,UKCP-RCM,land-rcm,15,-20.341466,FRA_S


# Write data to one file per season/region

In [4]:
# Write data to new file; one file per season/region combination
for region in regions:
    for season_number, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
        print(f"Writing data for {region}, {season}")
        subset = data.query("(region==@region)&(season_number==@season_number)")
        subset.drop(["season_number", "region"], axis=1).to_json(
            f"../static/data/{region}_{season}.json", orient="records"
        )

Writing data for Alpine, DJF
Writing data for Alpine, MAM
Writing data for Alpine, JJA
Writing data for Alpine, SON
Writing data for Alps_NE, DJF
Writing data for Alps_NE, MAM
Writing data for Alps_NE, JJA
Writing data for Alps_NE, SON
Writing data for Alps_NW, DJF
Writing data for Alps_NW, MAM
Writing data for Alps_NW, JJA
Writing data for Alps_NW, SON
Writing data for Alps_SE, DJF
Writing data for Alps_SE, MAM
Writing data for Alps_SE, JJA
Writing data for Alps_SE, SON
Writing data for Alps_SW, DJF
Writing data for Alps_SW, MAM
Writing data for Alps_SW, JJA
Writing data for Alps_SW, SON
Writing data for DE_S, DJF
Writing data for DE_S, MAM
Writing data for DE_S, JJA
Writing data for DE_S, SON
Writing data for FRA_S, DJF
Writing data for FRA_S, MAM
Writing data for FRA_S, JJA
Writing data for FRA_S, SON


# Generate vega spec / interactive figure

In [5]:
def plot(source):
    chart = (
        alt.Chart(source, width=75)
        .mark_circle(size=50)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("pr:Q"),
            color=alt.Color("project:N", legend=None),
            column=alt.Column(
                "project:N",
            ),
            tooltip=["dataset:N", "pr:Q"],
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=10)
        .configure_view(stroke=None)
    )
    return chart


# Save the chart with the data encoded as URL
url = "data/Alpine_DJF.csv"
chart = plot(url)
chart.save("../static/chart.json")

# Show the
print(chart.to_json())

# Show the figure using the live pandas dataframe
plot(subset)

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json",
  "config": {
    "facet": {
      "spacing": 10
    },
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400,
      "stroke": null
    }
  },
  "data": {
    "url": "data/Alpine_DJF.csv"
  },
  "encoding": {
    "color": {
      "field": "project",
      "legend": null,
      "type": "nominal"
    },
    "column": {
      "field": "project",
      "type": "nominal"
    },
    "tooltip": [
      {
        "field": "dataset",
        "type": "nominal"
      },
      {
        "field": "pr",
        "type": "quantitative"
      }
    ],
    "x": {
      "axis": {
        "grid": false,
        "labels": false,
        "ticks": true,
        "values": [
          0
        ]
      },
      "field": "jitter",
      "scale": {},
      "title": null,
      "type": "quantitative"
    },
    "y": {
      "field": "pr",
      "type": "quantitative"
    }
  },
  "mark": {
    "size": 50,
    "type": "circle

alt.Chart(...)